중첩 교차 검증(nested cross-validation)은 여러 종류의 머신러닝 알고리즘을 비교할때 유용하다.</br>
중첩 교차 검증은 바깥쪽 k-겹 교차 검증 루프가 훈련 폴드와 테스트 폴드를 나누고,</br>
안쪽 루프가 k-겹 교차 검증을 수행해서 모델을 선택한다.</br>
모델이 선택되면 테스트 폴드를 사용해서 모델의 성능을 평가한다.</br>

다음은 바깥 루프 5개 안쪽 루프에 두개 폴드를 사용한 중첩 교차 검증을 도식화한 것이다.

<img src="../../images/nested-cross-validation.png" width="700px" height="300px"/>
<caption><center>중첩 교차 검증</center></caption>

</br></br>
다음은 사이킷런의 중첩 교차 검증으로 SVC와 결정 트리의 성능을 비교한 것이다.

In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

df = pd.read_csv(   # 위스콘신 유방암 데이터셋 로딩
    'https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/wdbc.data', 
    header=None
)

x = df.loc[:, 2:].values    # 30개의 특성을 x에 할당
y = df.loc[:, 1].values     # 클래스 레이블을 y에 할당
y = LabelEncoder().fit_transform(y) # 클래스 레이블을 정수로 변환(악성: 1, 양성: 0)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, stratify=y, random_state=1)

In [3]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier

# 중첩 교차 검증으로 SVC 모델의 정확도 측정

param_range = [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0]
param_grid = [  # 튜닝 대상 매개변수의 리스트
    # 선형 SVM은 규제 매개변수 C만 튜닝한다.
    {'svc__C': param_range, 'svc__kernel': ['linear']},
    # RBF 커널 SVM에서는 C와 gamma 매개변수를 튜닝한다.
    {'svc__C': param_range, 'svc__gamma': param_range, 'svc__kernel': ['rbf']}
]
gs_svc = GridSearchCV(
    estimator=make_pipeline(StandardScaler(), SVC(random_state=1)), 
    param_grid=param_grid, 
    scoring='accuracy', 
    cv=2
)
scores_svc = cross_val_score(gs_svc, x_train, y_train, scoring='accuracy', cv=5)

print('SVC CV 정확도: %.3f +/- %.3f' % (np.mean(scores_svc), np.std(scores_svc)))

gs_dtc = GridSearchCV(
    estimator=DecisionTreeClassifier(random_state=0),
    param_grid=[{'max_depth': [1,2,3,4,5,6,7,None]}],
    scoring='accuracy',
    cv=2
)
scores_dtc = cross_val_score(gs_dtc, x_train, y_train, scoring='accuracy', cv=5)

print('결정 트리 CV 정확도: %.3f +/- %.3f' % (np.mean(scores_dtc), np.std(scores_dtc)))

SVC CV 정확도: 0.974 +/- 0.015
결정 트리 CV 정확도: 0.934 +/- 0.016


SVM 모델의 중첩 교차 검증 성능(97.4%)이 결정 트리의 성능(93.4%)보다 좋은 것을 알 수 있다.